In [1]:
import re

def segment_solidity_by_functions(code_str):
    lines = code_str.strip().splitlines()
    block_ranges = []
    function_starts = []
    brace_count = 0
    inside_function = False
    start_line = None

    for idx, line in enumerate(lines):
        stripped_line = line.strip()

        if not inside_function:
            if re.search(r'\bfunction\b|\bmodifier\b|\bconstructor\b', stripped_line) or re.search(r'\b\w+\s*\(', stripped_line):  # function or constructor
                if '{' in stripped_line:
                    brace_count = stripped_line.count('{') - stripped_line.count('}')
                    if brace_count > 0:
                        inside_function = True
                        start_line = idx
                else:
                    inside_function = True
                    start_line = idx
                    brace_count = 0
        else:
            brace_count += line.count('{') - line.count('}')
            if brace_count == 0:
                end_line = idx
                function_starts.append((start_line + 1, end_line + 1))  # 1-based index
                inside_function = False

    # Now generate blocks based on function boundaries
    last_end = 0
    for start, end in function_starts:
        block_start = last_end + 1
        block_end = end
        block_ranges.append((block_start, block_end))
        last_end = block_end

    return block_ranges


In [27]:
import sys
import pprint

from solidity_parser import parser

sourceUnit = parser.parse_file("18.sol", loc=True) # loc=True -> add location information to ast nodes
blocks = sourceUnit['children']


# print(blocks[2])
for block in blocks:
    try:
        print(block.get('loc', "NO START-END"))
    except:
        print("NO BLOCK")

{'start': {'line': 1, 'column': 0}, 'end': {'line': 10, 'column': 0}}
{'start': {'line': 18, 'column': 0}, 'end': {'line': 62, 'column': 0}}
{'start': {'line': 72, 'column': 0}, 'end': {'line': 91, 'column': 0}}
NO BLOCK
NO BLOCK
NO BLOCK
{'start': {'line': 96, 'column': 4}, 'end': {'line': 97, 'column': 1}}
{'start': {'line': 97, 'column': 4}, 'end': {'line': 97, 'column': 12}}


line 95:0 extraneous input '/' expecting {<EOF>, 'pragma', 'import', 'from', 'abstract', 'contract', 'interface', 'library', 'error', 'struct', 'function', 'enum', 'address', 'mapping', 'calldata', 'revert', 'var', 'bool', 'string', 'byte', 'callback', Int, Uint, Byte, Fixed, Ufixed, 'leave', 'payable', 'type', 'constructor', 'fallback', 'receive', Identifier}
line 96:3 token recognition error at: '@'
line 96:10 missing 'constant' at 'Ownable'
line 97:3 token recognition error at: '@'
line 97:1 mismatched input '*' expecting '='
line 97:8 missing 'constant' at 'The'
line 97:12 missing '=' at 'Ownable'
line 97:20 missing ';' at '<EOF>'


In [2]:
with open("18.sol", "r", encoding="utf-8") as f:
    code_text = f.read()

blocks = segment_solidity_by_functions(code_text)
for block in blocks:
    print("Block from line", block[0], "to", block[1])


Block from line 1 to 3
Block from line 4 to 6
Block from line 7 to 9
Block from line 10 to 34
Block from line 35 to 44
Block from line 45 to 48
Block from line 49 to 52
Block from line 53 to 61
Block from line 62 to 71
Block from line 72 to 75
Block from line 76 to 78
Block from line 79 to 88


In [4]:
print(code_text)

contract IMultiToken is IBasicMultiToken {
    event Update();
    event Change(address indexed _fromToken, address indexed _toToken, address indexed _changer, uint256 _amount, uint256 _return);

    function getReturn(address _fromToken, address _toToken, uint256 _amount) public view returns (uint256 returnAmount);
    function change(address _fromToken, address _toToken, uint256 _amount, uint256 _minReturn) public returns (uint256 returnAmount);

    function allWeights() public view returns(uint256[] _weights);
    function allTokensDecimalsBalancesWeights() public view returns(ERC20[] _tokens, uint8[] _decimals, uint256[] _balances, uint256[] _weights);
}

// File: openzeppelin-solidity/contracts/math/SafeMath.sol

/**
 * @title SafeMath
 * @dev Math operations with safety checks that throw on error
 */
library SafeMath {

  /**
  * @dev Multiplies two numbers, throws on overflow.
  */
  function mul(uint256 a, uint256 b) internal pure returns (uint256 c) {
    // Gas optimization: